In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
import pandas as pd
import numpy as np

In [55]:
target_value = '절도'
model_name = 'linear'
year = 2021

In [56]:
import pickle
with open('/content/drive/MyDrive/산업인공지능 대회/model/'+target_value+model_name+'.pickle', 'rb') as f:
    model = pickle.load(f)

In [57]:
df = pd.read_csv('/content/drive/MyDrive/산업인공지능 대회/merged_data/'+str(year)+'.csv', encoding='utf-8')

In [58]:
df

,위치,장소,요일,시간대,인구수,절도,살인,강도,성폭력,폭행
0,서울,아파트ㆍ연립ㆍ다세대,일,00:00-02:59,9509458,0.50,1.41,2.05,1.45,1.44
1,서울,아파트ㆍ연립ㆍ다세대,일,03:00-05:59,9509458,0.57,1.57,2.29,1.38,1.33
2,서울,아파트ㆍ연립ㆍ다세대,일,06:00-08:59,9509458,0.65,1.44,1.88,1.25,1.31
3,서울,아파트ㆍ연립ㆍ다세대,일,09:00-11:59,9509458,1.17,1.92,2.29,1.45,1.48
4,서울,아파트ㆍ연립ㆍ다세대,일,12:00-14:59,9509458,1.43,1.67,2.24,1.56,1.53
...,...,...,...,...,...,...,...,...,...,...
50171,도시이외,공장,토,09:00-11:59,3565452,1.67,0.93,0.91,1.19,1.25
50172,도시이외,공장,토,12:00-14:59,3565452,1.93,0.68,0.86,1.30,1.30
50173,도시이외,공장,토,15:00-17:59,3565452,2.00,0.99,0.92,1.45,1.37
50174,도시이외,공장,토,18:00-20:59,3565452,1.81,1.42,0.98,1.62,1.64


In [59]:
from sklearn.preprocessing import LabelEncoder
import numpy
import pickle
region = LabelEncoder()
region.classes_ = numpy.load('/content/drive/MyDrive/산업인공지능 대회/model/'+'region.npy', allow_pickle=True)
place = LabelEncoder()
place.classes_ = numpy.load('/content/drive/MyDrive/산업인공지능 대회/model/'+'place.npy', allow_pickle=True)
day = LabelEncoder()
day.classes_ = numpy.load('/content/drive/MyDrive/산업인공지능 대회/model/'+'day.npy', allow_pickle=True)
time = LabelEncoder()
time.classes_ = numpy.load('/content/drive/MyDrive/산업인공지능 대회/model/'+'time.npy', allow_pickle=True)

In [62]:
region.transform(['서울'])[0]

57

In [17]:
df['위치'] = region.transform(df['위치'])
df['장소'] = place.transform(df['장소'])
df['요일'] = day.transform(df['요일'])
df['시간대'] = time.transform(df['시간대'])

In [19]:
df

,위치,장소,요일,시간대,인구수,절도,살인,강도,성폭력,폭행
0,57,5,4,0,9509458,0.50,1.41,2.05,1.45,1.44
1,57,5,4,1,9509458,0.57,1.57,2.29,1.38,1.33
2,57,5,4,2,9509458,0.65,1.44,1.88,1.25,1.31
3,57,5,4,3,9509458,1.17,1.92,2.29,1.45,1.48
4,57,5,4,4,9509458,1.43,1.67,2.24,1.56,1.53
...,...,...,...,...,...,...,...,...,...,...
50171,36,0,5,3,3565452,1.67,0.93,0.91,1.19,1.25
50172,36,0,5,4,3565452,1.93,0.68,0.86,1.30,1.30
50173,36,0,5,5,3565452,2.00,0.99,0.92,1.45,1.37
50174,36,0,5,6,3565452,1.81,1.42,0.98,1.62,1.64


In [20]:
x_td = df.drop(['절도', '살인',	'강도',	'성폭력',	'폭행'], axis=1)
y_td = df[target_value]

In [24]:
x_y_mu_std = numpy.load('/content/drive/MyDrive/산업인공지능 대회/model/x_y_mu_std.npy',allow_pickle=True)

In [26]:
x_td_mu,x_td_std,y_td_mu,y_td_std = x_y_mu_std

In [27]:
def z_transform(X, mu, std):
    """
    ==========================
    | X: df, mu: df, std: df |
    ==========================
    """
    return (X-mu)/(std+0.000001)

def inv_z_transform(y, mu, std):
    """
    ====================================
    | y: ndarray, mu: df, std: df      |
    ====================================
    """
    return y * (std+0.000001) + mu

In [28]:
X = z_transform(x_td, x_td_mu, x_td_std)

In [29]:
pred_y = inv_z_transform(model.predict(X), y_td_mu, y_td_std)

In [32]:
error = y_td.values - pred_y

print('RMSE : ', np.sqrt((error**2).mean()))
print('MAE : ', np.mean(np.abs(error)))

RMSE :  0.5633327878068485
MAE :  0.4449735573287478


In [40]:
pred_y

array([0.6559085 , 0.78828945, 0.92067041, ..., 1.58745001, 1.71983097,
       1.85221192])

In [50]:
z_transform(np.array([57,	5,	4,	0,	9509458]), x_td_mu, x_td_std).values.reshape(1, -1)

array([[-0.17591619,  0.99999618,  0.49999809, -1.52751949,  8.31663546]])

In [49]:
np.array([57,	5,	4,	0,	9509458]).reshape(1,-1)

array([[     57,       5,       4,       0, 9509458]])

In [53]:
inv_z_transform(model.predict(z_transform(np.array([57,	5,	4,	0,	9509458]), x_td_mu, x_td_std).values.reshape(1, -1)), y_td_mu, y_td_std)[0]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


0.6559085005836665

### 다중 output

In [63]:
with open('/content/drive/MyDrive/산업인공지능 대회/model/linear.pickle', 'rb') as f:
    model = pickle.load(f)

In [64]:
df = pd.read_csv('/content/drive/MyDrive/산업인공지능 대회/merged_data/'+str(year)+'.csv', encoding='utf-8')

In [65]:
df

,위치,장소,요일,시간대,인구수,절도,살인,강도,성폭력,폭행
0,서울,아파트ㆍ연립ㆍ다세대,일,00:00-02:59,9509458,0.50,1.41,2.05,1.45,1.44
1,서울,아파트ㆍ연립ㆍ다세대,일,03:00-05:59,9509458,0.57,1.57,2.29,1.38,1.33
2,서울,아파트ㆍ연립ㆍ다세대,일,06:00-08:59,9509458,0.65,1.44,1.88,1.25,1.31
3,서울,아파트ㆍ연립ㆍ다세대,일,09:00-11:59,9509458,1.17,1.92,2.29,1.45,1.48
4,서울,아파트ㆍ연립ㆍ다세대,일,12:00-14:59,9509458,1.43,1.67,2.24,1.56,1.53
...,...,...,...,...,...,...,...,...,...,...
50171,도시이외,공장,토,09:00-11:59,3565452,1.67,0.93,0.91,1.19,1.25
50172,도시이외,공장,토,12:00-14:59,3565452,1.93,0.68,0.86,1.30,1.30
50173,도시이외,공장,토,15:00-17:59,3565452,2.00,0.99,0.92,1.45,1.37
50174,도시이외,공장,토,18:00-20:59,3565452,1.81,1.42,0.98,1.62,1.64


In [75]:
x_td_mu,x_td_std,y_td_mu,y_td_std = numpy.load('/content/drive/MyDrive/산업인공지능 대회/model/new_x_y_mu_std.npy',allow_pickle=True)

In [66]:
input = {
    "위치":"서울",
    "장소":"아파트ㆍ연립ㆍ다세대",
    "요일":"일",
    "시간대":"00:00-02:59",
    "인구수":9509458
}

In [78]:
X_new = np.array([
    int(region.transform([input["위치"]])[0]),
    int(place.transform([input["장소"]])[0]),
    int(day.transform([input["요일"]])[0]),
    int(time.transform([input["시간대"]])[0]),
    int(input["인구수"])
])
X_new = z_transform(X_new, x_td_mu, x_td_std).reshape(1, -1)
prediction = inv_z_transform(model.predict(X_new), y_td_mu, y_td_std)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [81]:
prediction.round(3)

array([[0.656, 1.021, 1.326, 1.054, 0.88 ]])